# Geospatial Data Analysis Lab: Steel Plants Dataset

by **Ulysse Mace** and **Daniil Notkin**

**(15/10/2025) Learning Objectives:**
- Perform exploratory data analysis (EDA) on geospatial datasets
- Visualize geospatial data using interactive maps with Plotly
- Merge environmental data with asset locations
- Aggregate data at the company level
- Integrate geospatial visualizations into a Streamlit dashboard

---


## Part 1: Setup and Data Loading

Import the necessary libraries and load the steel plants dataset.


In [1]:
# Import required libraries
# - pandas for data manipulation
# - numpy for numerical operations
# - plotly.express and plotly.graph_objects for interactive visualizations
# - Any other libraries you might need

import pandas as pd
import numpy as np
from plotly import express, graph_objects
import nbformat
import os
import plotly.express as px


In [2]:
# Load the steel plants dataset
# Expected columns: plant_id, plant_name, company, latitude, longitude, capacity, year_built, etc.
cur_dir = os.getcwd()
plant_dataset = pd.read_excel(f"{cur_dir}\\Plant-level-data-Global-Iron-and-Steel-Tracker-September-2025-V1.xlsx", sheet_name = "Plant data", na_values=["unknown", ">0", ">", ">2400"])

---
## Part 2: Exploratory Data Analysis (15 minutes)

Answer the following questions through your analysis:


### Question 1: Data Overview
**Task:** Display basic information about the dataset.
- How many steel plants are in the dataset?
- What are the column names and data types?
- Are there any missing values?


In [3]:
# Display dataset shape

print(plant_dataset.shape)

(1209, 44)


As we can see, we have 1209 rows and 44 columns, which represent the 1209 different steel plant IDs and their 44 individual characteristics

In [4]:
# Display column information and data types
print(plant_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 44 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Plant ID                            1209 non-null   object 
 1   Plant name (English)                1209 non-null   object 
 2   Plant name (other language)         697 non-null    object 
 3   Other plant names (English)         702 non-null    object 
 4   Other plant names (other language)  287 non-null    object 
 5   Owner                               1206 non-null   object 
 6   Owner (other language)              554 non-null    object 
 7   Owner GEM ID                        1209 non-null   object 
 8   Owner PermID                        724 non-null    object 
 9   SOE Status                          202 non-null    object 
 10  Parent                              1209 non-null   object 
 11  Parent GEM ID                       1209 no

In [5]:
# Check for missing values

print(plant_dataset.isna().sum())

Plant ID                                 0
Plant name (English)                     0
Plant name (other language)            512
Other plant names (English)            507
Other plant names (other language)     922
Owner                                    3
Owner (other language)                 655
Owner GEM ID                             0
Owner PermID                           485
SOE Status                            1007
Parent                                   0
Parent GEM ID                            0
Parent PermID                           20
Location address                        85
Municipality                           100
Subnational unit (province/state)       16
Country/Area                             0
Region                                   0
Other language location address        764
Coordinates                              0
Coordinate accuracy                      1
GEM wiki page                            0
Plant age (years)                      165
Announced d

From this analysis, we conclude that, depending on the column, we have a lot of missing values in the dataset (sometimes almost 1200), which means thaat it is far from being complete. There are however some columns that are quite detailed as they have little to no missing values.

- How many steel plants are in the dataset?

There are 1209 steel plants in the dataset, which corresponds to the 1209 different rows.

- What are the column names and data types?

The column's names and data types can be found above, with the code: print(plant_dataset.info())

- Are there any missing values?

Yes; There are missing values in the majority of the columns, and for some, more than half the values are missing.

### Question 2: Statistical Summary
**Task:** Generate descriptive statistics for numerical columns.
- What is the average plant capacity?
- What is the range of latitudes and longitudes?
- What is the distribution of plant ages?


In [6]:
# Display descriptive statistics

plant_dataset.describe()

,Plant age (years),Ferronickel capacity (ttpa),Sinter plant capacity (ttpa),Pelletizing plant capacity (ttpa),Workforce size
count,1044.000000,19.000000,179.000000,110.000000,967.000000
mean,39.532519,507.052632,4386.268156,3452.727273,3040.625646
std,37.021096,923.308392,3873.482293,4414.045913,4557.745949
min,0.000000,0.000000,0.000000,0.000000,2.000000
25%,16.000000,0.000000,1347.500000,1200.000000,532.500000
50%,25.240000,150.000000,3200.000000,2000.000000,1400.000000
75%,56.000000,300.000000,6066.000000,4425.000000,3500.000000
max,286.000000,3400.000000,18440.000000,28750.000000,38876.000000


In [7]:
plant_dataset["Coordinates"].describe()

count                        1209
unique                       1202
top       39.1604010, 118.4298060
freq                            2
Name: Coordinates, dtype: object

In [8]:
print(plant_dataset["Coordinates"])

0        41.0946110, 20.0261170
1         36.7539610, 6.2444200
2         36.8344120, 7.4549400
3        35.4785800, -0.6213900
4         36.7959240, 7.7076330
                 ...           
1204    10.5765970, 107.0289950
1205    20.4390250, 106.1621340
1206    20.0378050, 106.1276890
1207    -19.0551020, 29.7824240
1208    -18.9406580, 30.6089680
Name: Coordinates, Length: 1209, dtype: object


To get the proper range of the coordinates (max and min of the latitude and of the longitude), we need to split the coordinates column into two new columns; a latitude and a longitude column, which we can use to then get the min and max value of each, which will help us get the range.

In [9]:
#we split the Coordinates column into two new columns: Latitude and Longitude
plant_dataset[["Latitude", "Longitude"]] = (
    plant_dataset["Coordinates"]
    .str.split(",", expand=True)
    .apply(lambda x: x.str.strip())  #remove spaces to avoid bugs or glitches
)

#we convert the data to numeric (they’re strings by default)
plant_dataset["Latitude"] = pd.to_numeric(plant_dataset["Latitude"], errors="coerce")
plant_dataset["Longitude"] = pd.to_numeric(plant_dataset["Longitude"], errors="coerce")

#and now we get the maximum and minimum values separately for the range
max_lat = plant_dataset["Latitude"].max()
max_long = plant_dataset["Longitude"].max()
min_lat = plant_dataset["Latitude"].min()
min_long = plant_dataset["Longitude"].min()


print("Max Latitude:", max_lat)
print("Max Longitude:", max_long)
print("Min Latitude:", min_lat)
print("Min Longitude:", min_long)

Max Latitude: 67.189096
Max Longitude: 174.728098
Min Latitude: -37.831379
Min Longitude: -123.163599


- What is the average plant capacity?

    * The average ferronickel plant capacity is ~507
    * The average sinter plant capacity is ~4386
    * The average pelletizing plant capacity is ~3452

- What is the range of latitudes and longitudes?
    * The range of latitudes is -37.831379 to 67.189096
    * The range of longitudes is -123.163599 to 174.728098


- What is the distribution of plant ages?

The plants' ages range from 0 to 286 years. We derive this from the "Plant age (years)" column.

### Question 3: Geographic Distribution
**Task:** Analyze the geographic distribution of steel plants.
- Which countries/regions have the most steel plants?
- What is the distribution of plants by company?


In [10]:
# Count plants by country/region
print(plant_dataset["Country/Area"].value_counts())

Country/Area
China            404
India            108
United States     87
Iran              47
Japan             42
                ... 
Qatar              1
Sri Lanka          1
Slovenia           1
Singapore          1
Uganda             1
Name: count, Length: 89, dtype: int64


In [11]:
#we get the max country
top_country = plant_dataset["Country/Area"].value_counts().idxmax()
top_count = plant_dataset["Country/Area"].value_counts().max()

print("Country with most plants:", top_country)
print(f"Number of plants in {top_country}:", top_count)

Country with most plants: China
Number of plants in China: 404


In [12]:
# Count plants by company
plant_dataset["Owner"].value_counts()

Owner
Nucor Corp                                       13
Cleveland-Cliffs Inc                             12
Nippon Steel Corp                                10
Gerdau Ameristeel Corp                            8
Liberty Steel Group                               8
                                                 ..
Vina Roma Quang Tri Steel JSC                     1
VNSTEEL Southern Steel Co Ltd                     1
Baicheng Fuda Bar Rolling Co Ltd                  1
Berhard Development Corporation (Private) Ltd     1
Fortescue Ltd                                     1
Name: count, Length: 987, dtype: int64

- Which countries/regions have the most steel plants?

    * The country/area with the most steel plants is China, with 404 steel plants

- What is the distribution of plants by company?

    * We can find the number of plants each company has above, with the code plant_dataset["Owner"].value_counts()


### Question 4: Capacity Analysis
**Task:** Analyze the capacity distribution.
- What is the total global steel production capacity?
- Which companies have the highest total capacity?
- How does capacity vary by region?


In [13]:
# Calculate total capacity
# we do the sum of all plant's capacities and add them together
sinter = plant_dataset["Sinter plant capacity (ttpa)"].sum()
ferronickel = plant_dataset["Ferronickel capacity (ttpa)"].sum()
pelletizing = plant_dataset["Pelletizing plant capacity (ttpa)"].sum()

total_capacity = sinter+ferronickel+pelletizing

print(total_capacity)

1174576.0


The global steel production capacity is 1174576 ttpa

In [14]:
# Group by company and sum capacity
#we merge all capacity columns into one to do the company analysis
plant_dataset["Total capacity (ttpa)"] = (
    plant_dataset["Sinter plant capacity (ttpa)"].fillna(0)
    + plant_dataset["Ferronickel capacity (ttpa)"].fillna(0)
    + plant_dataset["Pelletizing plant capacity (ttpa)"].fillna(0)
)

#now we print the companies which have the highest total capacity
total_capacity_by_owner = (plant_dataset["Total capacity (ttpa)"].groupby(plant_dataset["Owner"], sort=False).sum().sort_values(ascending=False))

print(total_capacity_by_owner)


Owner
JSW Steel Ltd                                            39985.0
Jindal Steel Odisha Ltd                                  37500.0
Rungta Mines Ltd                                         33605.0
Jindal Steel & Power Ltd                                 33410.0
Steel Authority of India Ltd                             32091.0
                                                          ...   
Vina Roma Quang Tri Steel JSC                                0.0
VNSTEEL Southern Steel Co Ltd                                0.0
Xuan Thien Group JSC                                         0.0
Guangdong Jinshenglan Metallurgical Technology Co Ltd        0.0
Fortescue Ltd                                                0.0
Name: Total capacity (ttpa), Length: 987, dtype: float64


The 4 companies with the highest total capacity are
* JSW Steel Ltd
* Jindal Steel Odisha Ltd
* Rungta Mines Ltd
* Jindal Steel & Power Ltd.

In [15]:
#how capacity varies by region

total_capacity_by_country_area = (plant_dataset["Total capacity (ttpa)"].groupby(plant_dataset["Country/Area"], sort=False).sum().sort_values(ascending=False))
print(total_capacity_by_country_area)

Country/Area
India                   397429.0
China                   374743.0
Iran                     63850.0
Russia                   56828.0
Brazil                   40768.0
                          ...   
Syria                        0.0
Sweden                       0.0
Switzerland                  0.0
United Kingdom               0.0
United Arab Emirates         0.0
Name: Total capacity (ttpa), Length: 89, dtype: float64


Capacity is mostly focused on Asia and the Middle East, as the main 4 producers are:
* India (asia)
* China (asia)
* Iran (ME)
* Russia (Asia/Europe)

---
## Part 3: Geospatial Visualization with Plotly (15 minutes)

Create interactive maps to visualize the steel plants' locations and characteristics.


### Exercise 1: Basic Scatter Map
**Task:** Create a scatter map showing all steel plant locations.
- Use latitude and longitude for positioning
- Color points by country or region
- Add hover information showing plant name, company, and capacity


In [21]:
# Create a scatter_geo or scatter_mapbox plot
# Hint: Use plotly.express.scatter_geo() or scatter_mapbox()

#we start by creating the map to see how data can be visualized through express.scatter_geo()
express.scatter_geo(data_frame=plant_dataset, lat="Latitude", lon = "Longitude")

In [23]:
#and now we color it by country, and we add hover information showing the plant name (title), country of residence (country/area), company (owner), and total capacity (ttpa)

fig = px.scatter_geo(
    data_frame=plant_dataset,
    lat="Latitude",
    lon="Longitude",
    color="Country/Area",
    hover_name="Plant name (English)",
    hover_data={
        "Owner": True,
        "Total capacity (ttpa)": True,
        "Latitude": False,
        "Longitude": False
    },
    projection="natural earth",
    title="Global Steel Plants by Country",
)

fig.update_layout(
    legend_title_text="Country",
    geo=dict(showland=True, landcolor="lightgray", showcountries=True)
)

fig.show()

### Exercise 2: Sized Markers by Capacity
**Task:** Create a map where marker size represents plant capacity.
- Larger markers for higher capacity plants
- Color by company
- Include interactive hover details


We do a plot where plants with NaN values get their NaN replaced with the average value of all plants

In [27]:
plant_dataset_v1 = plant_dataset.copy(deep=True)

#ensure capacity column is numeric (just in case) 
plant_dataset_v1["Total capacity (ttpa)"] = pd.to_numeric(plant_dataset_v1["Total capacity (ttpa)"], errors="coerce")

#we fill NaNs in capacity with the mean of all plants
total_capacity_mean_v1 = plant_dataset_v1["Total capacity (ttpa)"].mean()
plant_dataset_v1["Total capacity (ttpa)"] = plant_dataset_v1["Total capacity (ttpa)"].fillna(total_capacity_mean_v1)

#just in case, we drop rows without coordinates (can't plot those)
plant_dataset_v1 = plant_dataset_v1.dropna(subset=["Latitude", "Longitude"])

In [ ]:
fig = px.scatter_geo(
    data_frame=plant_dataset_v1,
    lat="Latitude",
    lon="Longitude",
    color="Country/Area",size="Total capacity (ttpa)",size_max=22, hover_name="Plant name (English)",
    hover_data={
        "Owner": True,
        "Country/Area": True,
        "Total capacity (ttpa)": True,
        "Latitude": False,
        "Longitude": False
    },projection="natural earth",title="Global Steel Plants — Bubble Size by Total Capacity (NaNs filled with mean capacity)",)

fig.update_layout(legend_title_text="Region",geo=dict(showland=True, landcolor="lightgray", showcountries=True))

fig.show()

And now we do a plot where plants with NaN values get their NaN removed (0 / we drop the rows)

In [ ]:
plant_dataset_v2 = plant_dataset.copy(deep=True)
plant_dataset_v2 = plant_dataset_v2.dropna(subset=["Total capacity (ttpa)", "Latitude", "Longitude"])

In [ ]:
fig = px.scatter_geo(
    data_frame=plant_dataset_v2,lat="Latitude",lon="Longitude",color="Country/Area",size="Total capacity (ttpa)",size_max=22, hover_name="Plant name (English)",
    hover_data={
        "Owner": True,
        "Country/Area": True,
        "Total capacity (ttpa)": True
    },projection="natural earth",title="Global Steel Plants — NaN values Dropped",)

fig.update_layout(legend_title_text="Region", geo=dict(showland=True, landcolor="lightgray", showcountries=True))

fig.show()

The countries still show on the list in the right, but the plants do not appear on the map if they have zero (or NaN) production

### Exercise 3: Density Heatmap
**Task:** Create a density map showing concentration of steel plants.
- Use Plotly's density_mapbox to show clustering
- Identify regions with high plant density


In [35]:
# Create density heatmap
# Hint: Use plotly.express.density_mapbox()

#we drop nan values just in case they could cause issues
plants_geo = plant_dataset.dropna(subset=["Latitude", "Longitude"])


fig = px.density_mapbox(data_frame=plants_geo,lat="Latitude",lon="Longitude",radius=8,center={"lat": 20, "lon": 10},zoom=1,mapbox_style="open-street-map",title="Density Map — Global Steel Plant Concentration")

fig.show()

C:\Users\ulyss\AppData\Local\Temp\ipykernel_24520\3324937543.py:8: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



---
## ~~Part 4: Merging Environmental Data with Assets~~ skipped, due to large size of dataset

~~Integrate environmental data (e.g., air quality, emissions, proximity to water sources) with steel plant locations.~~

---
## Part 5: Company-Level Aggregation

Aggregate data at the company level to analyze corporate footprints.


### Exercise 1: Aggregate Metrics by Company
**Task:** Group plants by company and calculate aggregate metrics.
- Total capacity per company
- Number of plants per company
- Average environmental metrics per company
- Geographic spread (e.g., number of countries)


In [ ]:
# Group by company and aggregate
# in this case, I suppose that total capacity means sum total of all capacities

sinter_capacity = plant_dataset_b1["Sinter plant capacity (ttpa)"].groupby(plant_dataset_b1["Owner"]).sum()

### Exercise 2: Company Headquarters or Centroid
**Task:** Calculate a representative location for each company.
- Option 1: Use the centroid of all plant locations
- Option 2: Use the location of the largest plant
- Option 3: Assign actual headquarters coordinates


In [ ]:
# Calculate company representative locations



### Exercise 3: Visualize Company-Level Data
**Task:** Create a map showing companies with aggregated metrics.
- Show one marker per company at the representative location
- Size by total capacity
- Color by average environmental impact
- Hover information with company summary statistics


In [ ]:
# Create company-level visualization



---
## Part 6: Streamlit Dashboard Integration

Prepare your visualizations for deployment in a Streamlit dashboard.


### Exercise 1: Create Dashboard Script Structure
**Task:** Create a Streamlit app file (`dashboard.py`) with the following structure:

```python
# Import streamlit and other necessary libraries

# Set page configuration

# Title and description

# Sidebar for filters
# - Company selector
# - Region/country filter
# - Capacity range slider

# Main content area
# - KPI metrics (total plants, total capacity, etc.)
# - Interactive map
# - Data table

# Footer with data sources and notes
```


### Exercise 1: Prepare Data for Dashboard
**Task:** Save your processed data to files that the dashboard will load.
- Export cleaned plant data
- Export merged environmental data
- Export company-level aggregations
- Save as CSV or Parquet for efficient loading


In [ ]:
# Save processed datasets



### Exercise 2: Display relevant information from your exploratory analysis into the dashboard

In [ ]:
# This cell is for notes/observations about your dashboard
# What works well?
# What could be improved?
# Any performance issues with large datasets?



---
## Lab Summary and Key Takeaways

**What you learned:**
- How to perform EDA on geospatial datasets
- Creating interactive maps with Plotly for geospatial data
- Merging spatial datasets based on geographic proximity
- Aggregating geospatial data at different levels (asset vs. company)
- Building interactive dashboards with Streamlit

**Next Steps:**
- Explore other geospatial libraries (GeoPandas, Folium, Kepler.gl)
- Learn about coordinate reference systems (CRS) and projections
- Practice with other datasets (buildings, utilities, transportation)
- Deploy your dashboard to Streamlit Cloud or other hosting services
